In [1]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '0.1 alpha'
__keywords__ = ['astronomy', 'astropy', 'astroquery', 'matplotlib']


# tools for planning observations
## instruments tested : 
- 
- 